# Visualize filters and feature maps

## Set-up

In [ ]:
import zipfile
import os, os.path
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io
from pathlib import Path
import time
import copy

# import pytorch
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets, models

# to display filter and feater map images 
import matplotlib.pyplot as plt
# import cv2 as cv

In [ ]:
num_classes = 1000

## Loading Data

### Define Attribute Dictionary

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def print_opts(opts):
    """
    Print all the parameters in opts before training starts.
    """
    print('=' * 79)
    print('Opts'.center(79))
    print('-' * 79)
    for key in opts.__dict__:
        if opts.__dict__[key]:
            print('{:>30}: {:<30}'.format(key, opts.__dict__[key]).center(79))
    print('=' * 79)

### Define Data Loader

In [ ]:
def get_anime_loader(opts):
    """
    Retunrs a dictionary containing two key-value pairs:
        'train': training dataloader
        'test': validation dataloader
    """
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(opts.image_size),
            transforms.CenterCrop(opts.crop_size),
            transforms.RandomHorizontalFlip(p=opts.flip_prob),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(opts.image_size),
            transforms.CenterCrop(opts.crop_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    
    dataset_path = "dataset/"
    image_datasets = {category: datasets.ImageFolder(os.path.join(dataset_path, category),
                                                     data_transforms[category])
                      for category in ['train', 'test']}
    
    dataloader_dict = {category: torch.utils.data.DataLoader(image_datasets[category],
                                                         batch_size=opts.batch_size,
                                                         shuffle=True,
                                                         num_workers=opts.num_workers)
                   for category in ['train', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    print(f"dataset_sizes = {dataset_sizes}")
    print(f"Total number of images = {dataset_sizes['train'] + dataset_sizes['test']}")
    class_names = image_datasets['train'].classes
    # print(f"class_name 0 = {class_names[0]}")
    
    return dataloader_dict

## Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def googlenet_train_loop(model, criterion, optimizer, scheduler, epochs):
    
    model_weights = [[] for _ in range(epochs)] 
    conv_layers = [[] for _ in range(epochs)] 
    
    for epoch in range(epochs):
        train_loss = 0
        val_loss = 0
        accuracy = 0
        

        # Training the model
        model.train()
        counter = 0
        for inputs, labels in tqdm(dataloader_dict['train']):
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Clear optimizers
            optimizer.zero_grad()
            # Forward pass
            output, aux_outputs = model(inputs)
            # Loss
            loss = criterion(output, labels)
            # Calculate gradients (backpropogation)
            loss.backward()
            # Adjust parameters based on gradients
            optimizer.step()
            # Add the loss to the training set's rnning loss
            train_loss += loss.item() * inputs.size(0)
        # update learning rate scheduler
        scheduler.step()
        
        # Get the average loss for the entire epoch
        train_loss = train_loss/len(dataloader_dict['train'].dataset)

        # Print training loss
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        #################################################################################
        # copied from https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/#########
        # Refer to the [6] in reference section 
        model_children = list(model.children())
        for i in range(len(model_children)):
            if type(model_children[i]) == torchvision.models.inception.BasicConv2d:
                model_weights[epoch].append(model_children[i].conv.weight)
                # conv_layers[epoch].append(model_children[i].conv)
            # elif type(model_children[i]) == nn.Sequential:
            #     for j in range(len(model_children[i])):
            #         for child in model_children[i][j].children():
            #             if type(child) == nn.Conv2d:
            #                 model_weights[epoch].append(child.weight)
            #                 conv_layers[epoch].append(child)
        ##################################################################################

In [ ]:
def resnet_train_loop(model, criterion, optimizer, scheduler, epochs):
    
    model_weights = [[] for _ in range(epochs)] 
    conv_layers = [[] for _ in range(epochs)] 
    
    for epoch in range(epochs):
        train_loss = 0
        val_loss = 0
        accuracy = 0
        

        # Training the model
        model.train()
        counter = 0
        for inputs, labels in tqdm(dataloader_dict['train']):
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Clear optimizers
            optimizer.zero_grad()
            # Forward pass
            # output, aux_outputs = model(inputs)
            output = model(inputs)
            # Loss
            loss = criterion(output, labels)
            # Calculate gradients (backpropogation)
            loss.backward()
            # Adjust parameters based on gradients
            optimizer.step()
            # Add the loss to the training set's rnning loss
            train_loss += loss.item() * inputs.size(0)
        scheduler.step()


        # Get the average loss for the entire epoch
        train_loss = train_loss/len(dataloader_dict['train'].dataset)

        # Print training loss
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        #################################################################################
        # copied from https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/#########
        # Refer to the [6] in reference section
        # for each epoch, remember all weights
        
        model_children = list(model.children())
        for i in range(len(model_children)):
            if type(model_children[i]) == nn.Conv2d:
                model_weights[epoch].append(model_children[i].weight)
                conv_layers[epoch].append(model_children[i])
            elif type(model_children[i]) == nn.Sequential:
                for j in range(len(model_children[i])):
                    for child in model_children[i][j].children():
                        if type(child) == nn.Conv2d:
                            model_weights[epoch].append(child.weight)
                            conv_layers[epoch].append(child)
    return model_weights, conv_layers
        ##################################################################################

### Define Loss Function

In [ ]:
class AngularPenaltySMLoss(nn.Module):

    def __init__(self, in_features, out_features, loss_type='arcface', eps=1e-7, s=None, m=None):
        '''
        Angular Penalty Softmax Loss
        Three 'loss_types' available: ['arcface', 'sphereface', 'cosface']
        These losses are described in the following papers: 
        
        ArcFace: https://arxiv.org/abs/1801.07698
        SphereFace: https://arxiv.org/abs/1704.08063
        CosFace/Ad Margin: https://arxiv.org/abs/1801.05599
        '''
        super(AngularPenaltySMLoss, self).__init__()
        loss_type = loss_type.lower()
        assert loss_type in  ['arcface', 'sphereface', 'cosface']
        if loss_type == 'arcface':
            self.s = 64.0 if not s else s
            self.m = 0.5 if not m else m
        if loss_type == 'sphereface':
            self.s = 64.0 if not s else s
            self.m = 1.35 if not m else m
        if loss_type == 'cosface':
            self.s = 30.0 if not s else s
            self.m = 0.4 if not m else m
        self.loss_type = loss_type
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.eps = eps

    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels) < self.out_features
        
        for W in self.fc.parameters():
            W = F.normalize(W, p=2, dim=1)

        x = F.normalize(x, p=2, dim=1)

        wf = self.fc(x)
        if self.loss_type == 'cosface':
            numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        if self.loss_type == 'arcface':
            numerator = self.s * torch.cos(torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)) + self.m)
        if self.loss_type == 'sphereface':
            numerator = self.s * torch.cos(self.m * torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)))

        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L)

### Start Training

In [ ]:
def train(opts):
    train_model, criterion, optimizer, scheduler = None, None, None, None
    
    if opts.model == 'GoogLeNet':
        train_model = models.inception_v3(pretrained=True)
    else:
        train_model = models.resnet50(pretrained=True)
        
        
    for param in train_model.parameters():
        param.requires_grad = opts.requires_grad
        
    num_in_features = train_model.fc.in_features
    train_model.fc = nn.Linear(num_in_features, num_classes)
    
    # Handle the auxilary net
    # aux_num_ftrs = train_model.AuxLogits.fc.in_features
    # train_model.AuxLogits.fc = nn.Linear(aux_num_ftrs, num_classes)
    
    # Handle the primary net
    # primary_num_ftrs = train_model.fc.in_features
    # train_model.fc = nn.Linear(primary_num_ftrs, num_classes)
    
    if opts.loss_function == 'CrossEntropyLoss':
        train_model.fc = nn.Linear(num_in_features, num_classes)
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = AngularPenaltySMLoss(num_in_features, num_classes, loss_type=opts.loss_function)
        
    if opts.optimizer == 'SGD':
        optimizer = optim.SGD(train_model.parameters(), lr=opts.lr, momentum=0.9)
    else:
        optimizer = optim.Adam(train_model.parameters(), lr=opts.lr, betas=(0.9, 0.999), eps=1e-08)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=opts.step_size, gamma=opts.gamma)
    
    # move model to GPU
    train_model = train_model.to(device)
    print(f"Model moved to GPU: {device}")
    
    # model_weights conv_layers = googlenet_train_loop(train_model, criterion, optimizer, scheduler, opts.epochs)
    model_weights, conv_layers = resnet_train_loop(train_model, criterion, optimizer, scheduler, opts.epochs)
    return model_weights, conv_layers

### Define filter and feature map displaying function

In [ ]:
#################copied from https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/#########################
# Refer to the [6] in reference section
def displayFeatureMap(img_name, img, epoch): # which image? which epoch?
    results = [conv_layers[epoch][0](img)]
    for i in range(1, len(conv_layers[epoch])):
        # pass the result from the last layer to the next layer
        results.append(conv_layers[epoch][i](results[-1]))
    
    for num_layer in range(len(results)):
        plt.figure(figsize=(30, 30))
        layer_viz = results[num_layer][ :, :, :]
        layer_viz = layer_viz.data
        # print(layer_viz.size())
        for i, filter in enumerate(layer_viz):
            if i == 64: # we will visualize only 8x8 blocks from each layer
                break
            plt.subplot(8, 8, i + 1)
            plt.imshow(filter.cpu().clone().numpy(), cmap='gray')
            plt.axis("off")
        # print(f"Saving layer {num_layer} feature maps...")
        plt.savefig(f"outputs/img{img_name}-epoch{epoch}-layer{num_layer}.png")
        # plt.show()
        plt.close()
###################################################################################

In [ ]:
#################copied from https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/#########################
# Refer to the [6] in reference section
def displayFilter(epoch, layer):
    plt.figure(figsize=(20, 17))
    for i, filter in enumerate(model_weights[epoch][layer]):
        plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
        plt.imshow(filter[0, :, :].detach().cpu().clone().numpy(), cmap='gray')
        plt.axis('off')
        plt.savefig(f'outputs/filter-epoch{epoch}-layer{layer}.png')
    plt.show()
###########################################################################

In [ ]:
def transform_img(img_class, img_name, opts):
    """
    Given image class and image name, return a tuple ("img_class_img_name", tensor of the image)
    """
    transform_func = transforms.Compose([
        transforms.Resize(opts.image_size),
        transforms.CenterCrop(opts.crop_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    root_path = 'dataset/train'
    img_pil = Image.open(os.path.join(root_path, str(img_class), str(img_name) + ".jpg"))
    return "_".join([str(img_class), str(img_name)]), transform_func(img_pil).to(device)

In [ ]:
vis_args = AttrDict()
vis_args_dict = {
    'image_size': 256,  # 299
    'crop_size': 224,  # 299
}

vis_args.update(vis_args_dict)
transform_img(50, 6, vis_args)

('50_6',
 tensor([[[ 2.0777,  2.0777,  2.0777,  ...,  0.3309,  0.3309,  0.3652],
          [ 2.1119,  2.1119,  2.1119,  ...,  0.3309,  0.3309,  0.3309],
          [ 2.0948,  2.0948,  2.0948,  ...,  0.3309,  0.3309,  0.3309],
          ...,
          [-0.4397, -0.4397,  1.2728,  ...,  0.5878,  0.5878,  0.5878],
          [-0.4397, -0.4397,  1.3927,  ...,  0.5878,  0.5878,  0.5878],
          [-0.4397, -0.5424,  1.1700,  ...,  0.5878,  0.5878,  0.5878]],
 
         [[ 2.2710,  2.2710,  2.2710,  ...,  0.2052,  0.2052,  0.2402],
          [ 2.2360,  2.2360,  2.2360,  ...,  0.2052,  0.2052,  0.2052],
          [ 2.2185,  2.2185,  2.2185,  ...,  0.2052,  0.2052,  0.2052],
          ...,
          [-0.3200, -0.3200,  1.4307,  ...,  0.7304,  0.7304,  0.7304],
          [-0.3375, -0.3200,  1.5532,  ...,  0.7304,  0.7304,  0.7304],
          [-0.3375, -0.4426,  1.3081,  ...,  0.7304,  0.7304,  0.7304]],
 
         [[ 1.7163,  1.7163,  1.7163,  ..., -0.1138, -0.1138, -0.0790],
          [ 1.6988,

In [ ]:
args = AttrDict()
args_dict = {
    'image_size': 256,  # 299
    'crop_size': 224,  # 299
    'flip_prob': 0.3,
    'batch_size': 32,
    'num_workers': 2,
    'epochs': 10,
    'lr': 1e-3,
    'model': 'resnet',
    'loss_function': 'CrossEntropyLoss', # 'CrossEntropyLoss', 'arcface', 'sphereface', 'cosface'
    'optimizer': 'Adam',  # 'Adam', 'SGD'
    'step_size': 7,  # for learning rate scheduler
    'gamma': 0.1,  # for learning rate scheduler
    'requires_grad': False
}

args.update(args_dict)
print("Loading dataset...")
dataloader_dict = get_anime_loader(args)
print_opts(args)
model_weights, conv_layers = train(args)

Loading dataset...
dataset_sizes = {'train': 69849, 'test': 17435}
Total number of images = 87284
                                      Opts                                     
-------------------------------------------------------------------------------
                             image_size: 256                                   
                              crop_size: 224                                   
                              flip_prob: 0.3                                   
                             batch_size: 32                                    
                            num_workers: 2                                     
                                 epochs: 10                                    
                                     lr: 0.001                                 
                                  model: resnet                                
                          loss_function: CrossEntropyLoss                      
                      

100%|██████████| 2183/2183 [08:00<00:00,  4.54it/s]


Epoch: 0 	Training Loss: 3.102345


100%|██████████| 2183/2183 [08:04<00:00,  4.51it/s]


Epoch: 1 	Training Loss: 1.355407


100%|██████████| 2183/2183 [08:00<00:00,  4.54it/s]


Epoch: 2 	Training Loss: 1.043403


 42%|████▏     | 927/2183 [03:25<04:31,  4.63it/s]

In [ ]:
vis_args = AttrDict()
vis_args_dict = {
    'image_size': 256,  # 299
    'crop_size': 224,  # 299
}

vis_args.update(vis_args_dict)
img_class_img_name, tensor_of_the_image = transform_img(50, 6, vis_args)
displayFeatureMap(img_class_img_name, tensor_of_the_image, 0)